In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [ ]:
df = pd.read_csv("datasets/task_1_depression/depression_merged.csv")

In [ ]:
def to_liwc_token(text):
    for match in re.finditer(r'\w+', text, re.UNICODE):
        yield match.group(0).lower()

In [ ]:
import liwc
liwc_parse, categories = liwc.load_token_parser('dic/LIWC2007_English080730.dic')
K = len(categories)
kat_lookup = dict(zip(categories, range(K)))

In [ ]:
subjects = sorted(df.user.drop_duplicates().to_list())
S = len(subjects)
subj_lookup =  dict(zip(subjects, range(S)))
subj_ts = dict()

In [ ]:
# generate time serie
for subj, dfi in tqdm(df.groupby("user")):
    subj_ts[subj] = []

    for n, text in enumerate(dfi.sort_values("date_time").text):
        kat_freq = np.zeros(K)
        for t in to_liwc_token(text):
            for m in liwc_parse(t):
                k = kat_lookup[m]
                kat_freq[k] += 1

        if not kat_freq.sum():
            continue

        kat_freq /= kat_freq.sum()
        subj_ts[subj].append(kat_freq)
    subj_ts[subj] = np.array(subj_ts[subj])


In [ ]:
# filter out time series too small
MIN_LENGTH = 20
subj_ts = {k:v for k, v in subj_ts.items() if not v.shape[0] < MIN_LENGTH}

In [ ]:
"""
    classificatore sui amplificatoti nella trasformata di fourier (fft fast fourier)
"""